In [1]:
from __future__ import absolute_import, division, print_function
import tensorflow as tf
import tensorflow.contrib.eager as tfe
tf.enable_eager_execution()

즉시 실행 모드는 계산과정에서 자동으로 gpu와 분리 

In [2]:
import time

In [3]:
def measure(x, steps):
  # TensorFlow 최초 사용될때 GPU를 초기화 하므로, 시간 측정에서 제외시킵니다.
  tf.matmul(x, x)
  start = time.time()
  for i in range(steps):
    x = tf.matmul(x, x)
    _ = x.numpy()  # 연산만 수행 하도록 하고, 결과를 받아오는건 포함시키지 않도록 합니다.
  end = time.time()
  return end - start

In [4]:
shape = (1000, 1000)
steps = 200
print("Time to multiply a {} matrix by itself {} times:".format(shape, steps))

Time to multiply a (1000, 1000) matrix by itself 200 times:


In [5]:
# CPU에서 실행하기:
with tf.device("/cpu:0"):
  print("CPU: {} secs".format(measure(tf.random_normal(shape), steps)))

CPU: 1.9200997352600098 secs


In [6]:
# 가능하다면, GPU에서 실행하기:
if tfe.num_gpus() > 0:
  with tf.device("/gpu:0"):
    print("GPU: {} secs".format(measure(tf.random_normal(shape), steps)))
else:
  print("GPU: not found")

GPU: 0.3511321544647217 secs


### 그래프 환경에서 즉시 실행 사용하기 

ensorFlow 그래프 환경에서 tfe.py_func를 이용하여 선택적으로 즉시 실행 (eager execuution)을 활성화 시키세요. 하지만 tf.enable_eager_execution()이 호출되지 않았을 때만 사용가능 합니다.

In [7]:
def my_py_func(x):
  x = tf.matmul(x, x)  # tf 연산을 사용할 수 있습니다.
  print(x)  # 하지만 즉시 실행 모드로도 동작하죠!
  return x

In [8]:
with tf.Session() as sess:
  x = tf.placeholder(dtype=tf.float32)
  # Call eager function in graph!
  pf = tfe.py_func(my_py_func, [x], tf.float32)
  sess.run(pf, feed_dict={x: [[2.0]]})  # [[4.0]]

tf.Tensor([[4.]], shape=(1, 1), dtype=float32)
